In [1]:
import numpy as np

In [2]:
from nn_labs.nn.layers import BaseLayer
from nn_labs.nn.activation_functions import BaseActivationFunction

from numpy.typing import NDArray

class Network:
    def __init__(self):
        self.layers: list[BaseLayer | BaseActivationFunction] = []
    
    def add_layer(self, l: BaseLayer | BaseActivationFunction):
        self.layers.append(l)
        
    def forward(self, x_in: NDArray) -> NDArray:
        current = x_in
        
        for layer in self.layers:
            current = layer.forward(current)
            
        return current
    
    def backward(self, d_loss: NDArray):
        current_gradient = d_loss
        
        for layer in reversed(self.layers):
            # print(current_gradient.shape, layer.__class__)
            layer.backward(current_gradient)
            
            current_gradient = layer.d_inputs

In [3]:
from nn_labs.nn.layers import Conv2D, DenseLayer, Flatten, MaxPool2D
from nn_labs.nn.activation_functions import ReLU, Sigmoid

bob_net = Network()

bob_net.add_layer(Conv2D(1, 2, 16, 1))
bob_net.add_layer(MaxPool2D(2, 2))
bob_net.add_layer(Conv2D(16, 2, 16, 1))
bob_net.add_layer(MaxPool2D(2, 2))
bob_net.add_layer(Flatten())
bob_net.add_layer(DenseLayer(256, 400))
bob_net.add_layer(ReLU())
bob_net.add_layer(DenseLayer(400, 784))
bob_net.add_layer(Sigmoid())

In [4]:
x = np.ones((100, 1, 28, 28))

out = bob_net.forward(x)

out.shape

(100, 784)

In [ ]:
from nn_labs.nn.perf import MSELoss
from nn_labs.image_net.load import load_fashion_mnist
from nn_labs.nn.optimizers import PhilipOptimizer2

(X_train, Y_train), (X_test, Y_test) = load_fashion_mnist()

X_train = X_train.reshape(-1, 1, 28, 28)
X_test = X_test.reshape(-1, 1, 28, 28)

EPOCHS = 2000
BATCH_SIZE = 4086
LEARNING_RATE = 10e-3
batch_count = X_train.shape[0] // BATCH_SIZE
opt = PhilipOptimizer2(bob_net, learning_rate=1e-4)

loss_fn = MSELoss()

for epoch in range(EPOCHS):
    print(f"Epoch: {epoch}")
    for batch_no in range(batch_count):
        start_idx = batch_no * BATCH_SIZE
        end_idx = start_idx + BATCH_SIZE

        x = X_train[start_idx:end_idx]
        y = Y_train[start_idx:end_idx]
        
        x_cmp = x.reshape(BATCH_SIZE, 784)
        
        prediction = bob_net.forward(x)
        loss = loss_fn.calculate(prediction, x_cmp)
        
        print(loss)
        
        loss_fn.backward(prediction, x_cmp)
        bob_net.backward(loss_fn.d_inputs)
        
        opt.step()


Epoch: 0
0.17055302290657867
0.17060672382418676
0.170569420644888
0.16996645811468394
0.17084980822292786
0.17089439781323407
0.16990569976299558
0.1702694666313282
0.17036041409649869
0.17056123229944478
0.1697675684126144
0.17006187321667415
0.17045680084655399
0.1697574683058327
Epoch: 1
0.17029671899926907
0.17035313491455956
0.17031828547592334
0.16971798480646733
0.17060163163813208
0.17064631691231524
0.16966043078707874
0.17002473830615966
0.17011850981431847
0.1703197524625149
0.1695263492793537
0.16982011938407893
0.1702184504886373
0.16952051687933786
Epoch: 2
0.170056399768334
0.1701149351766252
0.17008196161963743
0.16948408654310695
0.1703680870770128
0.1704129988883193
0.16943025010905635
0.16979600651655938
0.16989365924987357
0.17009700051832535
0.16930556326659685
0.16960106301752662
0.17000542083344683
0.1693113594610256
Epoch: 3
0.16984748543449746
0.1699110754891323
0.16988290111655094
0.16929034145664315
0.1701778871798654
0.17022551412140982
